# Klue augmentation(x)

## 0. 라이브러리

In [91]:
import os
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, TFBertModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import json
from soynlp.normalizer import *
from tqdm import tqdm
import re
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

## 1. 데이터 불러오기

In [32]:
train_data_path ="../data2/train_add_2.csv"
train_data = pd.read_csv(train_data_path,index_col=0)
train_data.head(20)

,class,conversation
idx,,
1,일반 대화,오늘 날씨 어때?\n맑고 따뜻해.\n좋네! 주말에 계획 있어?\n등산 갈 생각이야....
2,일반 대화,오늘 날씨 어때?\n맑고 따뜻해.\n좋네! 주말에 계획 있어?\n등산 갈 생각이야....
3,기타 괴롭힘 대화,너 피아노 좀 쳐봐.\n싫어.\n왜 손가락 없다고 유세 떠는 거야?\n이씨.\n비행...
4,갈취 대화,요즘에는 액수가 작네?\n미안해 요즘에 용돈이 작아\n그게 나랑 무슨 상관이야?\n...
5,기타 괴롭힘 대화,야 이거봐 완전 길동이 닯음\n 진짜네 \n야 그러지마.\n왜 똑같구만 원숭이 \n...
6,갈취 대화,어이거기 \n저저희요?\n그래 거기 너네 둘\n왜왜요\n왜요?? 하참나 웃기는 놈들...
7,협박 대화,아 이새끼 대답안하는거 보소\n절대 안해\n아직도 말할 기운이 있나봐 ? 내가 좀 ...
8,협박 대화,시끄러워 살수가 없어요\n우리집은 아이도 교육시켜 조용히 걸어요\n지금도 아이 소리...
9,기타 괴롭힘 대화,새파랗게 젊은게 어디 여길 앉아있어\n저 임산부에요\n사지 멀쩡한게! 임신이 벼슬이...


In [33]:
print('훈련용 문단 개수 :',len(train_data))

훈련용 문단 개수 : 4950


In [34]:
train_data["class"].unique()

array(['일반 대화', '기타 괴롭힘 대화', '갈취 대화', '협박 대화', '직장 내 괴롭힘 대화'],
      dtype=object)

In [35]:
# class 별 데이터 내용 확인

In [36]:
print(train_data['conversation'].values[0]) # 일상대화

오늘 날씨 어때?
맑고 따뜻해.
좋네! 주말에 계획 있어?
등산 갈 생각이야.
재밌겠다!
최근에 본 영화 있어?
아직, 좋은 영화 있어?
정말 놀라워! 꼭 봐봐.
이번 주말에 볼게.
나중에 이야기하자.
좋아하는 책 뭐야?
'해리 포터' 정말 좋아해.
명작이지!
넌 어떤 책 좋아해?
난 '반지의 제왕' 팬이야.
보고서 다 했어?
응, 방금 제출했어.
완벽한 타이밍이야!
발표 준비는?
지금 작업 중이야.
수학 문제 좀 도와줄래?
물론, 어떤 문제야?
이 방정식에서 막혔어.
같이 풀어보자.
고마워, 정말 도움돼.


In [37]:
print(train_data['conversation'].values[2]) # 기타 괴롭힘 대화

너 피아노 좀 쳐봐.
싫어.
왜 손가락 없다고 유세 떠는 거야?
이씨.
비행기라도 쳐보라니까? 왜 못치지?
하지마.
너 내가 볼 땐 의지 문제야. 손가락 없다고 비행기를 못치냐.
흑.
신발 벗겨줄까? 발로 함 쳐보지 그래?
싫어.


In [38]:
print(train_data['conversation'].values[3]) # 갈취 대화

요즘에는 액수가 작네?
미안해 요즘에 용돈이 작아
그게 나랑 무슨 상관이야?
저번에 줬는데 또 필요해?
야야야 당연하지 너 지키는 값이 작냐?
아니 그건.
너 왕따에서 벗어나게 해준게 누구야 나잖아
늘 고맙게 생각하는데 이제 그만.
다시 왕따속에서 구르고 싶은가봐? 괴롭혀줄까?아니면 돈 가져올래?
. 다시 구해올게.


In [39]:
print(train_data['conversation'].values[6]) # 협박 대화

아 이새끼 대답안하는거 보소
절대 안해
아직도 말할 기운이 있나봐 ? 내가 좀 봐주면서 때렸나?
쿨럭
야 괜히 자존심 피우다가 지금보다 더 쳐맞는다?
그래도 싫어
하 어이가 옆에 야 잘잡아라 빗나가면 니네가 맞는다
 알겠어
그만해.
그만 하고 싶으면 내가 묻는 말에 대답 착실히 하면 되지?
그럴일은 없어.내가 여기서 나가면 니네 가만 안둘거야
아 진짜 아직도 입만 살아서 너는 죽어도 입만 물에 뜰것같다 오늘 한번 해볼까?입만 뜨는지?
여기서 멈춰줘


In [40]:
print(train_data['conversation'].values[16]) # 직장 내 괴롭힘 대화

야 우리 사장님 너무 인맥 위주로만 뽑지 않냐 다들 인성이 안된것 같아
야 새끼야 뒤에서 내 이야기를 뭘 그렇게 쑥덕거려 미친것아 너희 해고야!
아니 사장님 그렇다고 해고까지 주실 필요는 없잖아요
너 인맥 위주 싫다며 그런 사상도 안된놈이 뭔 우리 회사를 다녀
아니 사장님 제가 틀린말을 했나요? 너무 인맥 위주로만 뽑으니까 회사가 성적이 안나오죠
야 너 나한테 대들어 미친새끼야
아니 뭐 해고라니 그냥 당당히 노동청에다 알릴게요
알린다고 한들 너 주장 뿐인데 모든 증거는 다 입막음 되어있어 어디 전문변호사 고용해서 한판해봐?
죄송합니다.
입닥치고 넌 퇴직금도 없는줄 알아 완전 간이 부은 놈아니야


## 2. 데이터 준비 (Data preparation)
### 2.1-1 전처리 함수 정의

In [41]:
def preprocess_sentence(sentence):
    emoticon_normalize(sentence)
    repeat_normalize(sentence)
    sentence = re.sub(r'([^a-zA-Zㄱ-ㅎ가-힣?.!,])', " ", sentence)
    sentence = re.sub(r'!+', '!', sentence)
    sentence = re.sub(r'\?+', '?', sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.replace("\n", " ")
    sentence = sentence.strip()
    return sentence


### 2.1-2 전처리 함수 적용

In [42]:
# 학습할 문장이 담길 배열
sentences = []

for val in tqdm(train_data['conversation']):
    sentences.append(preprocess_sentence(val))


100%|██████████| 4950/4950 [00:01<00:00, 3194.02it/s]


### 2.2 최대 길이 지정

In [43]:
MAX_LEN = 200

### 2.3 class(label) 인코딩

In [44]:
from sklearn.preprocessing import LabelEncoder

CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화','일반 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])
labels = train_data['class']

len(labels)


4950

In [45]:
class_mapping = {class_name: encoder.transform([class_name])[0] for class_name in CLASS_NAMES}
print("Class mapping:", class_mapping)


Class mapping: {'협박 대화': 4, '갈취 대화': 0, '직장 내 괴롭힘 대화': 3, '기타 괴롭힘 대화': 1, '일반 대화': 2}


In [79]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(example, 
                                    max_length=max_seq_len, 
                                    pad_to_max_length=True,
                                   )
        
        # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        
        # token_type_id은 세그먼트 인코딩
        token_type_id = [0] * max_seq_len
        
        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)
    
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    
    data_labels = np.asarray(data_labels, dtype=np.int32)
    
    return (input_ids, attention_masks, token_type_ids), data_labels

### 2.4 train-val

In [104]:
X_train, X_valid, y_train, y_valid = train_test_split(
    sentences, labels, test_size=0.2, random_state=42)


## 3. 모델
### 3.1-1 토크나이저 정의

In [93]:
# klue model
model_name = "klue/bert-base"

In [94]:
# BERT 토크나이저와 모델 준비
tokenizer = BertTokenizer.from_pretrained(model_name)

### 3.1-2 토크나이저 적용

In [105]:
# 데이터셋을 BERT 입력 형식으로 변환
X_train, y_train = convert_examples_to_features(
    X_train, y_train, 
    max_seq_len=MAX_LEN, tokenizer=tokenizer
)
X_valid, y_valid = convert_examples_to_features(
    X_valid, y_valid, 
    max_seq_len=MAX_LEN, tokenizer=tokenizer
)

100%|██████████| 990/990 [00:00<00:00, 1014.71it/s]


In [97]:
class TFBertForMultiClassClassification(tf.keras.Model):
    def __init__(self, model_name, num_classes, dropout_rate=0.1):
        super(TFBertForMultiClassClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.dropout = Dropout(dropout_rate)
        self.classifier = tf.keras.layers.Dense(num_classes,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                kernel_regularizer=l2(0.01),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        cls_token = outputs[1]
        dropped = self.dropout(cls_token)
        prediction = self.classifier(dropped)
        return prediction

### 3.2 모델 준비

In [98]:
model = TFBertForMultiClassClassification(model_name, num_classes=5)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

### 3.3 파라미터

In [99]:
BATCH_SIZE = 16
lr = 5e-5
EPOCH = 10

### 3.4 TF 데이터셋 생성

In [106]:
# # TensorFlow 데이터셋 생성
# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_encodings),
#     train_labels
# )).shuffle(100).batch(BATCH_SIZE)

# val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(val_encodings),
#     val_labels
# )).batch(BATCH_SIZE)


### 3.5 모델 컴파일

In [109]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

### 3.6 모델 훈련

### 3.6-1 콜백 설정

In [110]:
early_stopping = EarlyStopping(
    monitor='val_loss',    # 검증 손실을 모니터링
    patience=3,            # 3 에포크 동안 개선되지 않으면 중지
    restore_best_weights=True  # 최상의 가중치를 복원
)

checkpoint = ModelCheckpoint(
    filepath='best_model_weights.h5',  # 모델 가중치를 저장할 파일 경로
    monitor='val_loss',        # 검증 손실을 모니터링
    save_best_only=True,       # 최상의 모델만 저장
    save_weights_only=True,   # 저장 (가중치)
    mode='min',                # 'val_loss'가 최소일 때 저장
    verbose=1                  # 저장 시 로그 출력
)


### 3.6-2 모델 훈련

In [111]:
model.fit(
    X_train, y_train, 
    validation_data=(X_valid, y_valid),
    epochs=EPOCH,
    callbacks=[early_stopping, checkpoint]
)


Epoch 1/10
124/124 [==============================] - 185s 1s/step - loss: 0.3751 - accuracy: 0.9038 - val_loss: 0.3139 - val_accuracy: 0.9162

Epoch 00001: val_loss improved from inf to 0.31388, saving model to best_model_weights.h5
Epoch 2/10
124/124 [==============================] - 168s 1s/step - loss: 0.1938 - accuracy: 0.9677 - val_loss: 0.3455 - val_accuracy: 0.9141

Epoch 00002: val_loss did not improve from 0.31388
Epoch 3/10
124/124 [==============================] - 168s 1s/step - loss: 0.1286 - accuracy: 0.9879 - val_loss: 0.4147 - val_accuracy: 0.9182

Epoch 00003: val_loss did not improve from 0.31388
Epoch 4/10
124/124 [==============================] - 168s 1s/step - loss: 0.1137 - accuracy: 0.9891 - val_loss: 0.5486 - val_accuracy: 0.8939

Epoch 00004: val_loss did not improve from 0.31388


### 3.7 모델 평가

In [114]:
# 모델 평가
evaluation = model.evaluate(X_valid, y_valid)
print("평가 결과:", evaluation)

31/31 [==============================] - 13s 428ms/step - loss: 0.3139 - accuracy: 0.9162
평가 결과: [0.31387799978256226, 0.9161615967750549]


### 3.8 결과 분석

In [112]:
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.metrics import accuracy_score

def score(model, val):
    X, y = val
    # 실제 예측값 생성
    real_predictions = model.predict(X)

    # 예측값을 레이블로 변환
    real_predicted_labels = np.argmax(real_predictions, axis=1)

    # 정확도 계산
    real_accuracy = accuracy_score(y, real_predicted_labels)
    print(f"Real Accuracy: {real_accuracy:.4f}")

    # 분류 보고서 생성
    real_report = classification_report(y, real_predicted_labels, target_names=[f"Class {i}" for i in range(5)])
    print(real_report)

    # F1 스코어 계산
    real_f1 = f1_score(y, real_predicted_labels, average='weighted')
    print(f"\nWeighted F1 Score (based on real predictions): {real_f1:.4f}")

In [115]:
score(model, (X_valid, y_valid))

Real Accuracy: 0.9162
              precision    recall  f1-score   support

     Class 0       0.84      0.90      0.87       191
     Class 1       0.88      0.91      0.89       223
     Class 2       1.00      1.00      1.00       191
     Class 3       0.96      0.95      0.96       200
     Class 4       0.90      0.82      0.86       185

    accuracy                           0.92       990
   macro avg       0.92      0.92      0.92       990
weighted avg       0.92      0.92      0.92       990


Weighted F1 Score (based on real predictions): 0.9161


## 4. 모델 적용

In [140]:
import json

test_data_path = "../data/test.json"

test = pd.read_json(test_data_path).transpose()

# with open(test_data_path, "r", encoding="utf-8") as json_file:
#     test = json.load(json_file)

In [141]:
print(test.head())

                                                    text
t_000  아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001  우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때?  네? 제가요? ...
t_002  너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_003  이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004  아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...


In [153]:
import numpy as np

test_predict = []

for idx, value in test.iterrows():

    test_sentence = value["text"]
    
    test_encodings = tokenizer(test_sentence, truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="tf")

    test_predictions = model.predict(
        (test_encodings["input_ids"],
            test_encodings["attention_mask"],
            test_encodings["token_type_ids"])
    )
    test_class_probabilities = tf.nn.softmax(test_predictions, axis=-1).numpy() # [[0.13297564 0.8358507  0.00801584 0.02315779]]
    test_predicted_class = np.argmax(test_class_probabilities, axis=1) # [ 1 ]
    test_predict.append(test_predicted_class[0])


In [154]:
print(len(test_predict))

500


In [155]:
print(test_predict[:5])

[0, 3, 3, 1, 1]


In [162]:
# {'협박 대화': 4, '갈취 대화': 0, '직장 내 괴롭힘 대화': 3, '기타 괴롭힘 대화': 1, '일반 대화': 2}

def labelnum_to_text(x):
    if x == 0:
        return '01'
    if x == 1:
        return '02'
    if x == 2:
        return '04'
    if x == 3:
        return '03'
    if x == 4:
        return '00'
    
submission = pd.DataFrame({"file_name":test.index, 'class':test_predict})

submission['class'] = submission['class'].apply(labelnum_to_text)
submission.head()

,file_name,class
0,t_000,01
1,t_001,03
2,t_002,03
3,t_003,02
4,t_004,02


In [163]:
submission.to_csv('../sub/base_sub.csv', index=False)

In [164]:
pd.read_csv("../sub/base_sub.csv").head()

,file_name,class
0,t_000,1
1,t_001,3
2,t_002,3
3,t_003,2
4,t_004,2
